# Initialize Packages


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from google.cloud import bigquery
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display

In [ ]:
client = bigquery.Client(project='ebce-models')

# Data Acquisition

In [ ]:
query = """
    -- 73MB
    SELECT *
    FROM `ebce-models.hdk_temp.weathersource_load_hourly_actuals`
"""

# Execute the query
query_job = client.query(query)

load_features = query_job.to_dataframe()

#Print dimensions
display(f"Load dimensions: {load_features.shape}")

# Display the DataFrame
load_features.head()

In [ ]:
# Plot number of customers, total system load vs datetime
# Create the figure
fig = go.Figure()

# Add trace for Total System Load
fig.add_trace(go.Scatter(
    x=load_features['datetime'], 
    y=load_features['total_system_load'], 
    mode='lines', 
    name='Total System Load', 
    line=dict(color='blue')
))

# Add trace for Number of Customers
fig.add_trace(go.Scatter(
    x=load_features['datetime'], 
    y=load_features['total_system_customers'], 
    mode='lines', 
    name='Number of Customers', 
    line=dict(color='green'),
    yaxis='y2'  # Specify secondary y-axis
))

# Update layout for two y-axes
fig.update_layout(
    title='Number of Customers and Total System Load Over Time',
    xaxis_title='Datetime',
    yaxis=dict(
        title='Total System Load',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title='Number of Customers',
        titlefont=dict(color='green'),
        tickfont=dict(color='green'),
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0, y=1),
    xaxis=dict(rangeslider=dict(visible=True)),
    template='plotly_white'
)

# Show the plot
fig.show()

# Feature Engineering

## Null Handling

In [ ]:
# Let's remove the one anomaly in 2023
load_features[(load_features['datetime'] >= '2023-03-06 22:00:00') & (load_features['datetime'] <= '2023-03-07 00:00:00')][['datetime', 'total_system_load', 'total_system_customers']]

In [ ]:
load_features_tx = load_features.copy()
load_features_tx  = load_features_tx[load_features_tx['datetime'] != '2023-03-06 23:00:00']
print(f"Load dimensions before transformation: {load_features.shape}")
print(f"Load dimensions after transformation: {load_features_tx.shape}")

In [ ]:
#Remove columns with data leakage
load_features_tx.drop(columns=['load_cz1', 'load_cz2', 'load_cz3', 'load_cz4', 'load_cz5', 'load_cz6'], inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)  
null_counts = load_features_tx.isnull().sum()
print(null_counts[null_counts > 0])
pd.set_option('display.max_rows', 10) 

In [ ]:
# Null Handling
#Set number of customers to 0 when that territory has no data.
cols_to_fill_zero = [
    'customers_cz2', 'customers_cz4', 'customers_cz5', 'customers_cz6'
]

# Set specific columns to 0 where they have null values
load_features_tx.loc[:, cols_to_fill_zero] = load_features_tx.loc[:, cols_to_fill_zero].fillna(0).copy()
print(f"Load dimensions before transformation: {load_features.shape}")
print(f"Load dimensions after transformation: {load_features_tx.shape}")

In [ ]:
# Coonvert to Sunday = 0, Monday = 1, Tuesday = 2, Wednesday = 3, Thursday = 4, Friday = 5, Saturday = 6

load_features_tx['day_of_week_mod'] = load_features_tx['datetime'].dt.weekday
# load_features_tx['day_of_week'].replace({7: 0})
load_features_tx[load_features_tx['datetime'].dt.hour == 0][['datetime', 'day_of_week', 'day_of_week_mod']].head(10)

## Holiday Handling

In [ ]:
# Holiday Handling
# Need to go back and handle surrounding days
load_features_tx['day_of_week'] = load_features_tx['datetime'].dt.weekday

# Convert 'datetime' to date
load_features_tx['date'] = load_features_tx['datetime'].dt.date
load_features_tx['day_of_week_modified'] = load_features_tx['day_of_week']

# Step 2: Handle holidays and surrounding days
# Assuming 'datetime' column is of datetime type
holidays = {
    'new_years_day': {'month': 1, 'day': 1, 'before': (12, 31), 'friday_to_saturday': True},
    'memorial_day': {'month': 5, 'day_last_monday': True, 'after': (5, 'last_monday')},
    'independence_day': {'month': 7, 'day': 4, 'before': (7, 3)},
    'labor_day': {'month': 9, 'day_first_monday': True, 'after': (9, 'first_monday')},
    'thanksgiving_day': {'month': 11, 'day_fourth_thursday': True, 'before': (11, 'fourth_thursday'), 'after': (11, 'fourth_thursday')},
    'christmas_day': {'month': 12, 'day': 25, 'before': (12, 24), 'after': (12, 26)}
}

# Helper functions to identify holidays
def is_holiday(date, holiday_info):
    if 'day' in holiday_info and date.month == holiday_info['month'] and date.day == holiday_info['day']:
        return True
    if 'before' in holiday_info and date.month == holiday_info['before'][0] and date.day == holiday_info['before'][1]:
        return True
    if 'after' in holiday_info and date.month == holiday_info['after'][0] and date.day == holiday_info['after'][1]:
        return True
    # Memorial Day
    if 'day_last_monday' in holiday_info and date.month == holiday_info['month'] and date.weekday() == 0:
        last_monday = max([d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(0)) if d.weekday() == 0])
        return date == last_monday
    # Day after memorial day
    if 'after' in holiday_info and date.month == holiday_info['month'] and holiday_info['after'][1] == 'last_monday' and date.weekday() == 1:
        last_monday = max([d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(0)) if d.weekday() == 0])
        return date == last_monday + 1 * pd.DateOffset(days=1)
    # Labor Day
    if 'day_first_monday' in holiday_info and date.month == holiday_info['month'] and date.weekday() == 0:
        first_monday = min([d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=f'{date.year}-{date.month}-07') if d.weekday() == 0])
        return date == first_monday   
    # Day after labor day
    if 'after' in holiday_info and date.month == holiday_info['month'] and holiday_info['after'][1] == 'first_monday' and date.weekday() == 1:
        first_monday = min([d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=f'{date.year}-{date.month}-07') if d.weekday() == 0])
        return date == first_monday + 1 * pd.DateOffset(days=1)
    # Day before Thanksgiving day
    if 'before' in holiday_info and date.month == holiday_info['month'] and holiday_info['before'][1] == 'fourth_thursday' and date.weekday() == 2:
        fourth_thursday = [d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(0)) if d.weekday() == 3][3]
        return date == fourth_thursday - 1 * pd.DateOffset(days=1)
    # Thanksgiving Day
    if 'day_fourth_thursday' in holiday_info and date.month == holiday_info['month'] and date.weekday() == 3:
        fourth_thursday = [d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(0)) if d.weekday() == 3][3]
        return date == fourth_thursday
    # Day after Thanksgiving day
    if 'after' in holiday_info and date.month == holiday_info['month'] and holiday_info['after'][1] == 'fourth_thursday' and date.weekday() == 4:
        fourth_thursday = [d for d in pd.date_range(start=f'{date.year}-{date.month}-01', end=pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(0)) if d.weekday() == 3][3]
        return date == fourth_thursday + 1 * pd.DateOffset(days=1) 
    return False

# Modify the 'day_of_week_modified' based on holidays
for index, row in load_features_tx.iterrows():
    date = row['datetime']
    for holiday, info in holidays.items():
        if is_holiday(date, info):
            if holiday in ('new_years_day', 'independence_day', 'christmas_day'):
                #Exact Holday
                if date.day == info['day']:
                    # If holiday is on Friday, move to Saturday
                    if date.weekday() == 4:  # Friday
                        load_features_tx.at[index, 'day_of_week_modified'] = 5  # Saturday
                    else:
                        load_features_tx.at[index, 'day_of_week_modified'] = 6  # Sunday
                # Handle day before independence day
                elif holiday == 'independence_day' and 'before' in info:
                    load_features_tx.at[index, 'day_of_week_modified'] = 4 # Friday
                # Day before or after holiday
                else:
                    load_features_tx.at[index, 'day_of_week_modified'] = 5 # Saturday
            elif holiday in ['memorial_day', 'labor_day', 'thanksgiving_day']:
                
                #Memorial Day
                if holiday == 'memorial_day' and date.weekday() == 0:
                    # print(date, holiday)
                    load_features_tx.at[index, 'day_of_week_modified'] = 5 # Saturday
                # Day after memorial day 
                elif holiday == 'memorial_day' and date.weekday() == 1:
                    # print(date, holiday)
                    load_features_tx.at[index, 'day_of_week_modified'] = 0 # Monday
                #Labor Day
                elif holiday == 'labor_day' and date.weekday() == 0:
                    load_features_tx.at[index, 'day_of_week_modified'] = 5 # Saturday
                # Day after labor day
                elif holiday == 'labor_day' and date.weekday() == 1:
                    load_features_tx.at[index, 'day_of_week_modified'] = 3 # Thursday
                # Day before Thanksgiving day
                elif holiday == 'thanksgiving_day' and date.weekday() == 2:
                    load_features_tx.at[index, 'day_of_week_modified'] = 0 # Monday
                #Thanksgiving Day
                elif holiday == 'thanksgiving_day' and date.weekday() == 3:
                    load_features_tx.at[index, 'day_of_week_modified'] = 5 # Saturday
                # Day after Thanksgiving day
                elif holiday == 'thanksgiving_day' and date.weekday() == 4:
                    load_features_tx.at[index, 'day_of_week_modified'] = 5 # Saturday

            # elif 'before' in info or 'after' in info:
            #     load_features_tx.at[index, 'day_of_week_modified'] = 6  # Saturday

# Convert Wednesdays to Tuesdays
load_features_tx['day_of_week_modified'] = load_features_tx['day_of_week_modified'].replace({2: 1})  # 2=Wednesday, 1=Tuesday

# Filter rows where the date matches the holiday list (ignoring the year)
holiday_examples = load_features_tx[load_features_tx['day_of_week'] != load_features_tx['day_of_week_modified']]

# Further filter to only show rows where the hour is 0 and not Tuesday
holiday_examples = holiday_examples[(holiday_examples['datetime'].dt.hour == 0) & (holiday_examples['day_of_week_modified'] != 1)]

# Display one example hour for each holiday
pd.set_option('display.max_rows', None)  
print(holiday_examples[['datetime', 'day_of_week', 'day_of_week_modified']].tail(25))
pd.set_option('display.max_rows', 10) 


## Periodic Temporal Feature Transformation

In [ ]:
# Transform month, day_of_year, and hour to cyclic features
load_features_tx['month_sin'] = np.sin(2 * np.pi * load_features_tx['month'] / 12)
load_features_tx['month_cos'] = np.cos(2 * np.pi * load_features_tx['month'] / 12)

load_features_tx['day_of_year_sin'] = np.sin(2 * np.pi * load_features_tx['day_of_year'] / 365)
load_features_tx['day_of_year_cos'] = np.cos(2 * np.pi * load_features_tx['day_of_year'] / 365)

load_features_tx['day_of_week_sin'] = np.sin(2 * np.pi * load_features_tx['day_of_week_modified'] / 7)
load_features_tx['day_of_week_cos'] = np.cos(2 * np.pi * load_features_tx['day_of_week_modified'] / 7)

load_features_tx['hour_sin'] = np.sin(2 * np.pi * load_features_tx['hour'] / 24)
load_features_tx['hour_cos'] = np.cos(2 * np.pi * load_features_tx['hour'] / 24)

# Multiple Linear Regression

In [ ]:
load_features_slim = load_features_tx[['datetime', 'total_system_load', 'total_system_customers', 
                                    # 'hdd_sensitivity', 'cdd_sensitivity'
                                    'temperature_avg', 'temp_1h_ago', 'temp_2h_ago', 'temp_3h_ago', 'avg_temp_last_24h',
                                    'hour', 'day_of_week_modified', 'month'
                                    ]]

# Add polynomial and cross effect terms
load_features_slim = load_features_slim.copy()  # Avoid SettingWithCopyWarning
load_features_slim['temperature_avg^2'] = load_features_slim['temperature_avg'] ** 2
load_features_slim['temperature_avg^3'] = load_features_slim['temperature_avg'] ** 3
load_features_slim['temp_1h_ago^2'] = load_features_slim['temp_1h_ago'] ** 2
load_features_slim['temp_1h_ago^3'] = load_features_slim['temp_1h_ago'] ** 3
load_features_slim['temp_2h_ago^2'] = load_features_slim['temp_2h_ago'] ** 2
load_features_slim['temp_2h_ago^3'] = load_features_slim['temp_2h_ago'] ** 3
load_features_slim['temp_3h_ago^2'] = load_features_slim['temp_3h_ago'] ** 2
load_features_slim['temp_3h_ago^3'] = load_features_slim['temp_3h_ago'] ** 3
load_features_slim['avg_temp_last_24h^2'] = load_features_slim['avg_temp_last_24h'] ** 2
load_features_slim['avg_temp_last_24h^3'] = load_features_slim['avg_temp_last_24h'] ** 3

# Function to add cross terms for indicator variables
def add_cross_terms(df, indicator_columns, numeric_columns):
    cross_term_data = []
    for ind_col in indicator_columns:
        for num_col in numeric_columns:
            cross_term_data.append(df[ind_col] * df[num_col])
            cross_term_data[-1].name = f'{ind_col}*{num_col}'
    return pd.concat([df] + cross_term_data, axis=1)

# Convert 'month' and 'day_of_week_modified' to indicator variables
hour_dummies = pd.get_dummies(load_features_slim['hour'], prefix='hour', drop_first=False)
month_dummies = pd.get_dummies(load_features_slim['month'], prefix='month', drop_first=False)
day_of_week_dummies = pd.get_dummies(load_features_slim['day_of_week_modified'], prefix='weekday', drop_first=False)

# Concatenate the dummies with the original dataframe
load_features_slim = pd.concat([load_features_slim, month_dummies, day_of_week_dummies, hour_dummies], axis=1)

# Get list of indicator and numeric columns
indicator_columns = month_dummies.columns.tolist() + hour_dummies.columns.tolist()
numeric_columns = ['temperature_avg', 'temperature_avg^2', 'temperature_avg^3', 
                   'temp_1h_ago', 'temp_1h_ago^2', 'temp_1h_ago^3',
                   'temp_2h_ago', 'temp_2h_ago^2', 'temp_2h_ago^3',
                   'temp_3h_ago', 'temp_3h_ago^2', 'temp_3h_ago^3',
                   'avg_temp_last_24h', 'avg_temp_last_24h^2', 'avg_temp_last_24h^3'
]

# Add cross terms for month and hour
load_features_slim = add_cross_terms(load_features_slim, indicator_columns, numeric_columns)

# Add cross terms for day of week and hour
load_features_slim = add_cross_terms(load_features_slim, day_of_week_dummies.columns.tolist(), month_dummies.columns.tolist())

#Drop columns that are not needed
load_features_slim.drop(columns=['hour', 'day_of_week_modified', 'month'], inplace=True)

load_features_slim.head()

## Time Filter
Hong 2014 uses only 2 years of training data. So we'll test on the most recent year and use the previous 2 years to train.

In [ ]:
# Limit to last 3 years
start_index = int(len(load_features_slim) - 365 * 24*3)  # Assuming data is hourly and each day has 24 hours
load_features_slim_time = load_features_slim.iloc[start_index:]
load_features_slim_time.shape

## Train Test Split

In [ ]:
# Define X (features) and y (target)
X = load_features_slim_time.drop(columns=['datetime', 'total_system_load'])
y = load_features_slim_time['total_system_load']

# Split data into training and validation sets (most recent year for validation)
# Assuming the data is in chronological order, the last year will be used for validation
split_index = int(len(X) - 365 * 24)  # Assuming data is hourly and each day has 24 hours
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Print the shapes to verify
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_test.shape}, y_val shape: {y_test.shape}")

## Train Model

## Training Set Error

In [ ]:
# Fit the Multiple Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_train)

# Evaluate the model
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
mape = np.mean(np.abs((y_train - y_pred) / y_train)) * 100

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Add back in the datetime for plotting
X_train_with_datetime = X_train.copy()
X_train_with_datetime['datetime'] = load_features_slim_time.loc[X_train.index, 'datetime']


# Plot y_pred vs y_val over time using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train_with_datetime['datetime'], y=y_train, mode='lines', name='Actual Load', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=X_train_with_datetime['datetime'], y=y_pred, mode='lines', name='Predicted Load', line=dict(color='red')))

fig.update_layout(
    title='Training Error:Predicted vs Actual Total System Load Over Time',
    xaxis_title='Datetime',
    yaxis_title='Total System Load',
    legend=dict(x=0, y=1),
    xaxis=dict(rangeslider=dict(visible=True)),
    template='plotly_white'
)

fig.show()


In [ ]:
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print("\nCoefficients of the model:")
print(coefficients.sort_values('Coefficient', ascending=False))

## Test Set Preds

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Add back in the datetime for plotting
X_test_with_datetime = X_test.copy()
X_test_with_datetime['datetime'] = load_features_slim_time.loc[X_test.index, 'datetime']


# Plot y_pred vs y_val over time using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test_with_datetime['datetime'], y=y_test, mode='lines', name='Actual Load', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=X_test_with_datetime['datetime'], y=y_pred, mode='lines', name='Predicted Load', line=dict(color='red')))

fig.update_layout(
    title='Predicted vs Actual Total System Load Over Time',
    xaxis_title='Datetime',
    yaxis_title='Total System Load',
    legend=dict(x=0, y=1),
    xaxis=dict(rangeslider=dict(visible=True)),
    template='plotly_white'
)

fig.show()



# Fully Connected Neural Network

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

## Hong 2014 Reduced Features Model

First, we experimented with training a NN on all 670 constructed features. 

In [ ]:
# Limit to last 3 years
start_index = int(len(load_features_slim) - 365 * 24*3)  # Assuming data is hourly and each day has 24 hours
load_features_slim_time = load_features_slim.iloc[start_index:]
load_features_slim_time.shape

In [ ]:
# Define X (features) and y (target)
X = load_features_slim_time.drop(columns=['datetime', 'total_system_load'])
y = load_features_slim_time['total_system_load']

# Split data into training and validation sets (most recent year for validation)
# Assuming the data is in chronological order, the last year will be used for validation
split_index = int(len(X) - 365 * 24)  # Assuming data is hourly and each day has 24 hours
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Print the shapes to verify
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_test.shape}, y_val shape: {y_test.shape}")

In [ ]:
model = Sequential([
    Dense(256, input_dim=X.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.3),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.2),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.1),
    
    Dense(1, activation='linear')
])

# Step 2: Compile the Model
# Lets try a faster learning rate
model.compile(optimizer=Adam(learning_rate=0.02), loss='mse', metrics=['mae', 'mape'])

In [ ]:
# Using the updated train and validation sets
def train_model(model, X_train, y_train, X_val, y_val):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
    ]
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=64, callbacks=callbacks)
    return history

In [ ]:
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

y_train_scaled.shape

In [ ]:
# Print shapes and ranges
print("Original y_train range:", y_train.min(), y_train.max())

# Scale and check
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("Scaled y_train range:", y_train_scaled.min(), y_train_scaled.max())
print("Scaled y_train mean:", y_train_scaled.mean())
print("Scaled y_train std:", y_train_scaled.std())

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)

# Also check the ranges
print("\nScaled ranges:")
print("X_train_scaled min/max:", X_train_scaled.min(), X_train_scaled.max())
print("y_train_scaled min/max:", y_train_scaled.min(), y_train_scaled.max())

In [ ]:
history = train_model(model, X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled)
# history = train_model(model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
# Step 5: Error Analysis - Visualize Predictions vs Actuals
y_pred_scaled = model.predict(X_test_scaled)

# Reverse the normalization of the target variable if needed
# y_val_actual = y_scaler.inverse_transform(y_val)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

# Create a DataFrame to hold actual vs predicted values
error_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})

# Add back in the datetime for plotting
# X_test_with_datetime = test_df_part2.copy()
error_df['datetime'] = load_features_slim_time.loc[error_df.index, 'datetime']

# Plot Actual vs Predicted using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=error_df['datetime'], y=error_df['Actual'], mode='lines', name='Actual Load'))
fig.add_trace(go.Scatter(x=error_df['datetime'], y=error_df['Predicted'], mode='lines', name='Predicted Load'))

fig.update_layout(
    title='Actual vs Predicted Load Over Time',
    xaxis_title='Datetime',
    yaxis_title='Load',
    legend=dict(x=0, y=1),
    xaxis=dict(rangeslider=dict(visible=True)),
    template='plotly_white'
)
fig.show()

In [ ]:
# Calculate MAPE for each point
error_df['MAPE'] = np.abs((error_df['Actual'] - error_df['Predicted']) / error_df['Actual']) * 100

# Create MAPE plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=error_df['datetime'], 
    y=error_df['MAPE'], 
    mode='lines', 
    name='MAPE'
))

fig.update_layout(
    title='MAPE Over Time',
    xaxis_title='Datetime',
    yaxis_title='MAPE (%)',
    template='plotly_white',
    showlegend=True,
    xaxis=dict(rangeslider=dict(visible=False))
)

# Add a line for average MAPE
avg_mape = error_df['MAPE'].mean()
fig.add_hline(y=avg_mape, line_dash="dash", line_color="red", 
              annotation_text=f"Avg MAPE: {avg_mape:.2f}%")

fig.show()

# Print summary statistics of MAPE
print("\nMAPE Statistics:")
print(error_df['MAPE'].describe())

Training a FCNN with 670 features, we obtain a MAPE of ~17.3%, well above our MAPE from linear regression. Let's transtion to training a FCNN with our base features.

## Period Temporal Features Model

### Feature Engineering

In [ ]:
core_features = ['datetime', 'total_system_load', 'total_system_customers']
time_features = ['month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos', 'hour_sin', 'hour_cos']
temp_features = ['temperature_avg', 'temp_1h_ago', 'temp_2h_ago', 'temp_3h_ago', 'avg_temp_last_24h']

load_features_nn = load_features_tx[core_features + time_features + temp_features]
print(load_features_nn.shape)
load_features_nn.columns

In [ ]:
load_features_nn.tail()

### Train/ Validation/ Test Split

Here we split our data into 2 years of training data, one year of validation data to choose the optimal parameters, and one year of test data. Prior to running the model on test data, we will retrain the model on the two years preceeding the test data.

In [ ]:
# Ensure your data is sorted by datetime
load_features_nn = load_features_nn.sort_values('datetime')

# Convert the 'datetime' column to pandas datetime format if it's not already
load_features_nn['datetime'] = pd.to_datetime(load_features_nn['datetime'])

# Define X (features) and y (target)
X = load_features_nn.drop(columns=['datetime', 'total_system_load'])
y = load_features_nn['total_system_load']

# Step 1: Hyperparameter Tuning
# Split data by specific date ranges
train_df_part1 = load_features_nn[(load_features_nn['datetime'] >= '2020-11-01') & (load_features_nn['datetime'] < '2022-11-01')]
val_df_part1 = load_features_nn[(load_features_nn['datetime'] >= '2022-11-01') & (load_features_nn['datetime'] < '2023-11-01')]

X_train_part1 = train_df_part1.drop(columns=['datetime', 'total_system_load'])
y_train_part1 = train_df_part1['total_system_load']
X_val_part1 = val_df_part1.drop(columns=['datetime', 'total_system_load'])
y_val_part1 = val_df_part1['total_system_load']

print("Part 1 - Hyperparameter Tuning:")
print(f"X_train_part1 shape: {X_train_part1.shape}, y_train_part1 shape: {y_train_part1.shape}")
print(f"X_val_part1 shape: {X_val_part1.shape}, y_val_part1 shape: {y_val_part1.shape}")

# Scale the features and target
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit and transform the training set
X_train_part1_scaled = scaler_X.fit_transform(X_train_part1)
y_train_part1_scaled = scaler_y.fit_transform(y_train_part1.values.reshape(-1, 1))

# Transform the validation set using the same scaler
X_val_part1_scaled = scaler_X.transform(X_val_part1)
y_val_part1_scaled = scaler_y.transform(y_val_part1.values.reshape(-1, 1))

# Part 2: Final Training and Testing
train_df_part2 = load_features_nn[(load_features_nn['datetime'] >= '2021-11-01') & (load_features_nn['datetime'] < '2023-11-01')]
test_df_part2 = load_features_nn[(load_features_nn['datetime'] >= '2023-11-01') & (load_features_nn['datetime'] < '2024-11-01')]

X_train_part2 = train_df_part2.drop(columns=['datetime', 'total_system_load'])
y_train_part2 = train_df_part2['total_system_load']
X_test_part2 = test_df_part2.drop(columns=['datetime', 'total_system_load'])
y_test_part2 = test_df_part2['total_system_load']

print("\nPart 2 - Final Training and Testing:")
print(f"X_train_part2 shape: {X_train_part2.shape}, y_train_part2 shape: {y_train_part2.shape}")
print(f"X_test_part2 shape: {X_test_part2.shape}, y_test_part2 shape: {y_test_part2.shape}")

# Scale the features and target using the same scalers from Part 1
X_train_part2_scaled = scaler_X.transform(X_train_part2)
y_train_part2_scaled = scaler_y.transform(y_train_part2.values.reshape(-1, 1)) # should I flatten this?

X_test_part2_scaled = scaler_X.transform(X_test_part2)
y_test_part2_scaled = scaler_y.transform(y_test_part2.values.reshape(-1, 1)) #Should I flatten this?

In [ ]:
print("Shapes After Scaling:")
print(f"X_train_part1_scaled: {X_train_part1_scaled.shape}, y_train_part1_scaled: {y_train_part1_scaled.shape}")
print(f"X_val_part1_scaled: {X_val_part1_scaled.shape}, y_val_part1_scaled: {y_val_part1_scaled.shape}")
print(f"X_train_part2_scaled: {X_train_part2_scaled.shape}, y_train_part2_scaled: {y_train_part2_scaled.shape}")
print(f"X_test_part2_scaled: {X_test_part2_scaled.shape}, y_test_part2_scaled: {y_test_part2_scaled.shape}")


### Train Model

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import mean_absolute_percentage_error

#Print Scaled MAPE every epoch
class RealTimeMAPECallback(Callback):
    def __init__(self, X_val, y_val, y_scaler):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.y_scaler = y_scaler

    def on_epoch_end(self, epoch, logs=None):
        # Get predictions on validation data
        y_pred_scaled = self.model.predict(self.X_val)
        
        # Inverse transform predictions and actuals
        y_pred = self.y_scaler.inverse_transform(y_pred_scaled)
        y_true = self.y_scaler.inverse_transform(self.y_val)
        
        # Calculate MAPE
        mape = mean_absolute_percentage_error(y_true, y_pred) * 100
        print(f"Epoch {epoch + 1} - Real-time MAPE on validation data: {mape:.3f}%")

In [ ]:
model = Sequential([
    Dense(256, input_dim=X.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.2),
    
#     Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
#     BatchNormalization(),
#     Dropout(0.2),
    
#     Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
#     BatchNormalization(),
#     Dropout(0.2),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.1),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    # Dropout(0.1),
    
    Dense(1, activation='linear')
])

# Step 2: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae', 'mape'])

In [ ]:



# Using the updated train and validation sets
def train_model(model, X_train, y_train, X_val, y_val, y_scaler):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-6),
        RealTimeMAPECallback(X_val, y_val, y_scaler)
    ]
    history = model.fit(X_train, 
                        y_train, 
                        validation_data=(X_val, y_val), 
                        epochs=100, 
                        batch_size=64,  # Power of 2 near sqrt(n_samples)
                        callbacks=callbacks)
    return history

In [ ]:
history = train_model(model, X_train_part2_scaled, y_train_part2_scaled, X_test_part2_scaled, y_test_part2_scaled, y_scaler)

### Cross Validation

In [ ]:
import keras_tuner as kt
from tensorflow.keras import Sequential, layers
from sklearn.model_selection import RandomizedSearchCV

Here we perform cross validation to determine the optimal hyperparameters. In order to optimize the batch size, we needed to modify the keras tuner class.

In [ ]:
# Predefined layer configurations
layer_options = [
    [256, 128, 16],
    [128, 64, 32],
    [64, 32, 16],
    [32, 16, 8]
]

dropout_options = [
    [None, None, None], 
    [0.2, 0.2, 0.1], 
    [0.3, 0.2, 0.1]
]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        # Define layers using hyperparameters
        layer_index = hp.Choice('layers', [0, 1, 2, 3])  # 0: [256, 128, 64], 1: [256, 256, 128], 2: [128, 64, 32]
        layers_config = layer_options[layer_index]
        
        # Let's not use dropout now since we are using batch normalization
        # dropout_index = hp.Choice('dropout_rates', [0, 1, 2])
        # dropout_rates = dropout_options[dropout_index]

        learning_rate = hp.Choice('learning_rate', [0.0005, 0.001, 0.005, 0.01, 0.02])
        # l2_reg = hp.Choice('l2_reg', [0.001, 0.005, 0.01])

        # Build the model
        model = Sequential()
        for i, units in enumerate(layers_config):
            model.add(layers.Dense(units, activation='relu'
                                   # , kernel_regularizer=l2(l2_reg) # Remove because we have batch norm
                                  ))
            model.add(layers.BatchNormalization())
            # if dropout_rates[i]:
            #     model.add(layers.Dropout(dropout_rates[i]))

        # Output layer
        model.add(layers.Dense(1, activation='linear'))
        
        # Compile the model
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae', 'mape'])
        return model

    def fit(self, hp, model, *args, **kwargs):
        # Include batch_size as a tunable hyperparameter during training
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [32, 64, 128]),
            **kwargs,
        )

In [ ]:


# Function to perform hyperparameter tuning
def perform_hyperparameter_tuning(X_train, y_train, X_val, y_val):
    global tuner
    tuner = kt.RandomSearch(
        MyHyperModel(),
        objective='val_loss',
        # max_epochs=100,
        max_trials=30,
        # factor=3,
        directory='random_search', #'hyperband_limited_search',
        project_name='load_forecasting_with_random_search',
        overwrite=True
    )
    
    stop_early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Perform the search using the custom validation set
    tuner.search(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        callbacks=[stop_early],
        verbose=1
    )
    
    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_hps


In [ ]:
best_hps = perform_hyperparameter_tuning(
    X_train_part1_scaled, y_train_part1_scaled, 
    X_val_part1_scaled, y_val_part1_scaled
)
print(f"Best hyperparameters: {best_hps.values}")

In [ ]:
best_hps.values['batch_size']

In [ ]:
import pandas as pd

def display_all_trials(tuner):
    # Extract all trials
    trials = tuner.oracle.trials.values()
    
    # Prepare a list to store the results
    trial_data = []
    
    # Loop through each trial and extract relevant information
    for trial in trials:
        # Get the history and extract the last step (epoch) number
        history = trial.metrics.get_history('val_loss')
        epochs_run = history[-1].step if history else 'N/A'
        
        trial_info = {
            'Trial ID': trial.trial_id,
            'Layers': trial.hyperparameters.get('layers'),
            # 'Dropout Rates': trial.hyperparameters.get('dropout_rates'),
            # 'Activation': trial.hyperparameters.get('activation'),
            'Learning Rate': trial.hyperparameters.get('learning_rate'),
            # 'L2 Regularization': trial.hyperparameters.get('l2_reg'),
            'Batch Size': trial.hyperparameters.get('batch_size'),
            'Num epochs': epochs_run,
            'Validation Loss': trial.score
        }
        trial_data.append(trial_info)
    
    # Convert the results to a pandas DataFrame for better visualization
    df = pd.DataFrame(trial_data)
    
    # Sort by validation loss (lower is better)
    df.sort_values(by='Validation Loss', ascending=True, inplace=True)
    return df

# Call the function to display all the trials
trials_df = display_all_trials(tuner)

pd.set_option('display.max_rows', None)  
print(trials_df)
pd.set_option('display.max_rows', 10) 

In [ ]:
trials = tuner.oracle.trials.values()
for trial in trials:
    print(trial.metrics.get_history('val_loss'))

In [ ]:
#Continue training best model

def create_model(input_dim, layers, learning_rate):
    model = Sequential([
        Dense(layers[0], input_dim=input_dim, activation='relu'),
        BatchNormalization(),
        Dense(layers[1], activation='relu'),
        BatchNormalization(),
        Dense(layers[2], activation='relu'),
        BatchNormalization(),
        Dense(1, activation='linear')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae', 'mape']
    )
    return model

def train_model(model, X_train, y_train, X_val, y_val, y_scaler, batch_size=64, epochs=100):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-6),
        RealTimeMAPECallback(X_val, y_val, y_scaler)
    ]
    return model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]


model = create_model(
    input_dim=X_train_part1_scaled.shape[1],
    layers=layer_options[best_hps.get('layers')],
    learning_rate=best_hps.get('learning_rate')
)

history = train_model(
    model, 
    X_train_part1_scaled, 
    y_train_part1_scaled, 
    X_val_part1_scaled, 
    y_val_part1_scaled, 
    y_scaler,
    batch_size=best_hps.get('batch_size')
)

### Retrain on 2022-2023 data

Using the parameters previous determined, let's retrain on 2022-2023 data to build a model that we will use to predict on 2024 data. This ensures the same recency relationship between train and test data. 

In [ ]:

# Use them like this:
model = create_model(
    input_dim=X_train_part2_scaled.shape[1],
    layers=layer_options[0], #layer_options[best_hps.get('layers')],
    learning_rate=0.02 #best_hps.get('learning_rate')
)

history = train_model(
    model, 
    X_train_part2_scaled, 
    y_train_part2_scaled, 
    X_test_part2_scaled, 
    y_test_part2_scaled, 
    y_scaler,
    batch_size=64 # best_hps.get('batch_size')
)

In [ ]:
# Step 4: Visualize the Metrics
# Plot the training and validation loss
plt.figure(figsize=(14, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot MAPE
plt.subplot(1, 2, 2)
plt.plot(history.history['mape'], label='Training MAPE')
plt.plot(history.history['val_mape'], label='Validation MAPE')
plt.title('Model MAPE')
plt.xlabel('Epoch')
plt.ylabel('MAPE (%)')
plt.legend()

plt.show()

In [ ]:
# Step 4: Visualize the Metrics with Plotly
# Plot the training and validation loss
fig = go.Figure()

# Add traces for loss
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))

fig.update_layout(
    title='Model Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)
fig.show()

# Plot the MAPE
fig = go.Figure()

# Add traces for MAPE
fig.add_trace(go.Scatter(y=history.history['mape'], mode='lines', name='Training MAPE'))
fig.add_trace(go.Scatter(y=history.history['val_mape'], mode='lines', name='Validation MAPE'))

fig.update_layout(
    title='Model MAPE',
    xaxis_title='Epoch',
    yaxis_title='MAPE (%)',
    hovermode='x unified'
)
fig.show()

### Test Set Preds

In [ ]:
# Make predictions on the validation set
y_pred = model.predict(X_test_part2_scaled)

In [ ]:
# y_pred = y_pred.reshape(-1)
y_pred.shape

In [ ]:
# Step 5: Error Analysis - Visualize Predictions vs Actuals
y_pred_scaled = model.predict(X_test_part2_scaled)

# Reverse the normalization of the target variable if needed
# y_val_actual = y_scaler.inverse_transform(y_val)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

# Create a DataFrame to hold actual vs predicted values
error_df = pd.DataFrame({'Actual': y_test_part2, 'Predicted': y_pred.flatten()})

# Add back in the datetime for plotting
# X_test_with_datetime = test_df_part2.copy()
error_df['datetime'] = test_df_part2.loc[error_df.index, 'datetime']

# Plot Actual vs Predicted using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=error_df['datetime'], y=error_df['Actual'], mode='lines', name='Actual Load'))
fig.add_trace(go.Scatter(x=error_df['datetime'], y=error_df['Predicted'], mode='lines', name='Predicted Load'))

fig.update_layout(
    title='Actual vs Predicted Load Over Time',
    xaxis_title='Datetime',
    yaxis_title='Load',
    legend=dict(x=0, y=1),
    xaxis=dict(rangeslider=dict(visible=True)),
    template='plotly_white'
)
fig.show()

### Plot MAPE

In [ ]:
# Calculate MAPE for each point
error_df['MAPE'] = np.abs((error_df['Actual'] - error_df['Predicted']) / error_df['Actual']) * 100

# Create MAPE plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=error_df['datetime'], 
    y=error_df['MAPE'], 
    mode='lines', 
    name='MAPE'
))

fig.update_layout(
    title='MAPE Over Time',
    xaxis_title='Datetime',
    yaxis_title='MAPE (%)',
    template='plotly_white',
    showlegend=True,
    xaxis=dict(rangeslider=dict(visible=False))
)

# Add a line for average MAPE
avg_mape = error_df['MAPE'].mean()
fig.add_hline(y=avg_mape, line_dash="dash", line_color="red", 
              annotation_text=f"Avg MAPE: {avg_mape:.2f}%")

fig.show()

# Print summary statistics of MAPE
print("\nMAPE Statistics:")
print(error_df['MAPE'].describe())

Now, we observe a significantly lower MAPE of ~4.35 that beats both the previous 670 NN model and the linear regression model.